In [15]:
import os
import sys
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

In [16]:
class DQN(nn.Module):
    def __init__(self, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(24, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)
        torch.nn.init.uniform(self.fc3.weight, -1e-3, 1e-3)
        # 텐서 값을 특정 법위에서 균일하게 초기화하는 함수, 신경망의 가중치 초기화
        # Why? 가중치가 잘못 초기화되면 학습이 제대로 이루어지지 않거나 느려질 수 있음.
        # 비교적 작은 값을 가지도록 설정하여 기울기 소실, 기울기 폭발 방지, 안정적인 학습 가능


    def call(self, x): # forward
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(X))
        q = self.fc3(x)
        return q

In [17]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.render = False

        # 상태와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        # DQN 하이퍼파라미터
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.train_start = 1000

        # 리플레이 메모리, 최대 크기 2000
        self.memory = deque(maxlen=2000)

        # 모델과 타깃 모델 생성
        self.model = DQN(action_size)
        self.target_model = DQN(action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

        # 타깃 모델 초기화
        self.update_target_model()

    # 타깃 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        else:
            q_value = self.model(state)
            return np.argmax(q_value)

    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        if self.epsilon > self.epsilon_min:
            self.epilon *= self.epsilon_decay

            # 메모리에서 배치 크기만큼 무작위로 샘플 추출ㄹ (미니배치 경사 하강법)
            # 한 번의 샘플이 아닌 배치 크기만큼 추출하는 이유 :
            # 한 번의 샘플만 사용하면 가중치가 특정 데이터에 의해 크게 변할 수 있음.
            mini_batch = random.sample(self.memery, self.batch_size)

            states = np.array([sample[0][0] for sample in mini_batch])
            actions = np.array([sample[1] for sample in mini_batch])
            rewards = np.array([sample[2] for sample in mini_batch])
            next_states = np.array([sample[3][0] for sample in mini_batch])
            dones = np.array([sample[4] for sample in mini_batch])

            # 학습 파라미터
            model_params = self.model.parameters()


            # 현재 상태에 대한 모델의 큐함수
            predicts = self.model(states) # DQN 네트워크의 출력값 (Q-values)
            one_hot_action = F.one_hot(actions, num_classes=action_size)
            # actions : 정수형 텐서, 각 샘플에서 취한 행동
            predicts = torch.sum(one_hot_action * predicts, dim=1)

            # 다음 상태에 대한 타깃 모델의 큐함수
            with torch.no_grad():
                target_predicts = self.target_model(next_states)

            # 벨만 최적 방정식을 이용한 업데이트 타깃
            max_q = np.amax(target_predicts, axis=1)
            targets = rewards + (1 - dones) * self.gamma * max_q


            loss = self.criterion(targets - predicts)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()



In [18]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # DQN 에이전트 생성
    agent = DQNAgent(state_size, action_size)

    scores, episodes = [], []
    score_avg = 0

    num_episode = 300
    for e in range(num_episode):
        done = False
        score = 0
        # env 초기화
        state = env.rest()
        state = np.reshape(state, [1, state_size]) #?

        while not done:
            if agent.render:
                env.render()

            # 현재 상태로 행동을 선택
            action = agent.get_action(state)
            # 선택한 행동으로 환경에서 한 타임스텝 진행
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # 타임스텝마다 보상 0.1, 에피소드 중간에 끝나면 -1 보상
            score += reward
            reward = 0.1 if not done or score == 500 else -1

            # 리플레이 메모리에 샘플 <s, a, r, s'> 저장
            agent.append_sample(state, action, reward, next_state, done)
            # 타임스텝마다 학습
            if len(agent.memeory) >= agent.train_start:
                agent.train_model()

            state = next_state

            if done:
                # 각 에피소드마다 타깃 모델을 모델의 가중치로 업데이트
                agent.update_target_model()
                # 에피소드마다 학습 결과 출력
                score_avg = 0.9 * score_avg * 0.1 * score if score_avg != 0 else score
                print("episode: {:3d} | score avg: {:3.2f} | memory length: {:4d} | epsilon: {:.4f}".format(
                    e, score, len(agent.memory), agent.epsilon))

                # 에피소드마다 학습 결과 그래프로 저장
                scores.append(score_avg)
                episodes.append(e)
                plt.ploy(episodes, scores, 'b')
                plt.xlabel('Episode')
                plt.ylabel('average score')

                # 이동 평균이 400 이상일 때 종료
                if score_avg > 400:
                    torch.save(agent.model.state_dict(), ",/save_model/model.pth")
                    break


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
<ipython-input-16-3d0530fd7128>:7: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.fc3.weight, -1e-3, 1e-3)


AttributeError: 'DQN' object has no attribute 'set_weights'